# Patsy contrast tutorial

The following tutorial fro the patsy package is based on Schad, Vasishth, Hohenstein, and Kliegl (2020), Tutorial of contrast coding for analysis specification.

In [2]:
# Import packages for conducting the tutorial
import pandas as pd 
import numpy as np 
import seaborn as sns
from statsmodels.formula.api import ols

# The General issue

The data produced from experimental designs are often analysed using some variant of the analysis of variance (ANOVA) depnding on the experimental design. Standard practive is analsye the data with the desired ANOVA check the F-test for significance followed by Post hoc analysis of all the differences with some form pairwise comparison (Bonferroni, typically). This apporaoch is limited though if researchers have a prior theory driven comparison hypotheses before seeing the data.

The tutorial below focuses on Frequentist statistics but its application in the following notebooks is of course Bayesian.

In [3]:
# Simulate data for two groups
np.random.seed(1)
F1 = np.random.normal(0.8,0.2,5)
F2 = np.random.normal(0.4,0.2,5)
n = 5

data = {'Subject': range(n),
        'F1': F1,
         'F2': F2
        }

df = pd.DataFrame (data, columns = ['Subject','F1','F2'])
df =pd.melt(df,id_vars=['Subject'],var_name='F', value_name='DV')
df["Subject"] = df["Subject"] + 1
df

,Subject,F,DV
0,1,F1,1.124869
1,2,F1,0.677649
2,3,F1,0.694366
3,4,F1,0.585406
4,5,F1,0.973082
5,1,F2,-0.060308
6,2,F2,0.748962
7,3,F2,0.247759
8,4,F2,0.463808
9,5,F2,0.350126


In [4]:
mod = ols("DV ~ F", data=df)
res = mod.fit()

print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                     DV   R-squared:                       0.488
Model:                            OLS   Adj. R-squared:                  0.424
Method:                 Least Squares   F-statistic:                     7.618
Date:                Thu, 19 Nov 2020   Prob (F-statistic):             0.0247
Time:                        13:01:08   Log-Likelihood:                0.24060
No. Observations:                  10   AIC:                             3.519
Df Residuals:                       8   BIC:                             4.124
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.8111      0.118      6.867      0.0

C:\Users\harri\Anaconda3\lib\site-packages\scipy\stats\stats.py:1450: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=10
  "anyway, n=%i" % int(n))



## $$ Intercept (F1) = \hat{\mu}_1 = 0.81$$
## $$ Slope (F2) = \hat{\mu}_1 - \hat{\mu}_2 = -0.46$$

When using Treatment coding the mean of baseline/control group is the intercept of the ouptut and the Slope (Beta) is the diffence between the two groups, please note that tis was all generated by defaukt in the OLS function, so if not specifying the exact cotrasts be careful with interpretation.

# Genral linear model formulation

Why these regression coeffiecnt are produced 

# Treatment Contrasts
The default contrast in many ststisticsl softwars is the treatment contrast. This type of statisical model contrast name i derived from its general use in medical settings, where treatments are compared to a baseline (control group). (insert example here).

array([ 26.24345364,   3.88243586,   4.71828248,  -0.72968622,
        18.65407629, -13.01538697,  27.44811764,   2.38793099,
        13.19039096,   7.50629625])

# Sum contrats

This type of test contrast is different from the treatment contrasts, as the diffence is generated by comparing eaxch facors mean against the grand mean of all the levels of the factor. 

In [34]:
print(range(1,n))

range(1, 5)


# Repeated Contrasts

This contrast compares groups in succesive order. i.e if you had groups separated by some level of manipulation with settings of low, medium and high; low would be compared to medium and then medium compared to high.

# Citations

Schad, D. J., Vasishth, S., Hohenstein, S., & Kliegl, R. (2020). How to capitalize on a priori contrasts in linear (mixed) models: A tutorial. Journal of Memory and Language, 110, 104038.